In [1]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━

In [2]:
from google.colab import output

output.clear()

In [3]:
import os

from google.colab import userdata

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [4]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# 데이터 불러오기

In [5]:
bl_data_sampled=pd.read_csv('bl_sample_data.csv')

display(bl_data_sampled.head())

,HSCD,SUPLY_CO_NAME,PRDT_DC_VAL
0,330499,"MESOESTETIC,. S.L.",BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM)
1,330499,"MESOESTETIC,. S.L.",COSMELAN 2 (FACE CREAM)
2,330499,"MESOESTETIC,. S.L.",ANTI STRESS FACE MASK 100ML (FACE CREAM)
3,330499,KRYOLAN GMBH CHEMISCHE FABRIK,BLOT POWDER
4,330499,KRYOLAN GMBH CHEMISCHE FABRIK,DERMACOLOR CAMOUFLAGE FLUID


In [6]:
bk_data_sampled=pd.read_excel('BK_330499_수정.xlsx')

display(bk_data_sampled.head())

,NO,PID,CORPNAME,PRD_NAME,PRD,URL
0,1,3723376,"CELLVIO COSMETIC CO.,LTD.",Doldori Premium Gold Essence,Our patented mask pack redefines skincare with...,https://buykorea.org/ec/prd/selectGoodsDetail....
1,2,3722918,Brade,Exosome Exo-V Skin Boos,The EXO-V Exosome Skin Booster is used in Kore...,https://buykorea.org/ec/prd/selectGoodsDetail....
2,3,3722956,"BIOVAIKOREA Co.,LTD",Oil-Free Ultra-Moisturizing Lotion with Birch Sap,DETAILS\nBENEFITS\n\n\n• Soothing irritated sk...,https://buykorea.org/ec/prd/selectGoodsDetail....
3,4,3722734,Hit Company,CHARNE - Shiny Brightening Cream,Charne Shiny Brightening Cream\n\nVarious natu...,https://buykorea.org/ec/prd/selectGoodsDetail....
4,5,3717513,"Wooanjoo Co.,Ltd",DEAR OHNEUL Singreen Donkey Toner Pad,Product Features:\n\n－ Hypoallergenic: Dermato...,https://buykorea.org/ec/prd/selectGoodsDetail....


# Langsmith , LLM API KEY 설정

In [7]:
from langchain_teddynote import logging

# logging.langsmith("KOTRA_BL_data_matching")

In [8]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


# API 사용 rate 제한 설정


In [9]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=5,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-9-5cc4985cb9b0>:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_openai = InMemoryRateLimiter(


# 사용 모델 설정

In [10]:
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)

# OpenAI API 이용 결과 (model : chatgpt 4o-mini)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 업무 배경 설명 (제품명, 상품 설명에 대한 데이터임을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 제품명과 상품 정보에 기반한 제품 설명 생성
+ 지시 내용별 예시 제공 (one-shot)

In [61]:
keyword_num=5

class DescriptionSummary(BaseModel):
    type : str = Field(description="product type")
    keyword : list[str] = Field(description=f"{keyword_num} keywords which focus on product's strong point and purpose")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

In [23]:
summary_prompt_buykorea = PromptTemplate.from_template(
    """You are given data about product's description(PRDT_DESC) and product name(PRDT_NAME).
    All products fall under HS Code 330499, which is a subcategory of 3304.
    The description for 3304 is as follows:
    33.04 - Beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or sun tan preparations; manicure or pedicure preparations.

    Using this information, complete the following tasks:
    Provide a simple summary for description.

    Example :
    PRDT_DESC : 'FIORESE Pure Hydra Water Cream contains Centella Asiatica and Hyaluronic Acid, effectively delivering hydration and providing instant soothing benefits.
    With Niacinamide and Adenosine, it is a dual-functional cosmetic for brightening and wrinkle care. This cream helps create healthy, smooth, and radiant skin.
    The lightweight, hydrating formula absorbs quickly, delivering moisture deep into the skin with a fresh, non-sticky finish. Suitable for all skin types, especially sensitive, dull, and uneven skin.

    For Sensitive Skin: Gentle, irritation-free formula, suitable even for sensitive skin.

    FIORESE Brand: Fiorese is dedicated to clean beauty, using ingredients sourced from nature. 100% cruelty-free.'

    PRDT_COM : DAMOA COSMETICS.CO.LTD

    Output Example : The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients.

    PRODUCT DESCRIPTION(PRDT_DESC) : {prdt_desc}
    PRODUCT NAME(PRDT_NAME) : {prdt_name}
    """
)


summary_chain_buykorea = summary_prompt_buykorea | chat_openai

In [34]:
buykorea_preprocessed_dict_openai={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = summary_chain_buykorea.invoke({"prdt_desc": row['PRD'], "prdt_name": row['PRD_NAME']})
  print(response_buykorea.content)
  buykorea_preprocessed_dict_openai[row['PRD_NAME']]=[response_buykorea.content]

The Doldori Premium Gold Essence is a unique and innovative mask pack designed for a luxurious skincare experience. Its patented, hygienic design offers a new application method that combines practicality with elegance, making it stand out in the market. This mask pack is available for export and is sold in aesthetic clinics, department store pop-ups, and online shopping malls in Korea, reflecting its high demand.

The product is manufactured with meticulous attention to quality, from the essence formulation to the mask sheet development. Enriched with high-quality ingredients, the essence hydrates, brightens, and revitalizes the skin, while the mask sheet is designed for excellent adhesion, ensuring comfort and deep absorption of the essence. The Doldori Premium Gold Essence delivers visible skincare benefits along with an indulgent experience, making it a luxurious addition to any skincare routine.
The Exosome Exo-V Skin Booster is a versatile skin treatment used in top plastic surge

- Strictly exclude adjectives, adverbs, prepositions, .

In [35]:
extract_prompt_buykorea = PromptTemplate.from_template(
    """Extract product type and keywords from input, based on following rules:
    - Strictly exclude any information about capacity or volume.

    INPUT : {input}
    FORMAT : {format}
    """
)

extract_prompt_buykorea= extract_prompt_buykorea.partial(format=parser.get_format_instructions())

extract_chain_buykorea = extract_prompt_buykorea | chat_openai | parser

In [36]:
for key, value in buykorea_preprocessed_dict_openai.items():
  response_buykorea = extract_chain_buykorea.invoke({"input": value})
  print(response_buykorea)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.type)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.keyword)

type='mask pack' keyword=['luxurious skincare', 'hygienic design', 'high-quality ingredients', 'hydration', 'skin revitalization']
type='Skin Treatment' keyword=['exosomes', 'skin regeneration', 'anti-aging', 'moisture retention', 'wrinkle reduction']
type='lotion' keyword=['oil-free', 'moisturizing', 'soothing', 'hydration', 'lightweight']
type='cosmetic cream' keyword=['skin whitening', 'wrinkle improvement', 'natural ingredients', 'antioxidant benefits', 'skin vitality']
type='Toner Pad' keyword=['hypoallergenic', 'dual-sided', 'moisturizing', 'eco-friendly', 'pore improvement']
type='skincare product' keyword=['soothing gel', 'hydration', 'snail mucin', 'watermelon extract', 'antioxidants']
type='Sunscreen' keyword=['Vegan', 'Non-Nano', 'UV Protection', 'Hypoallergenic', 'Environmentally Friendly']
type='moisturizing cream' keyword=['moisturizing', 'hydrating', 'squalane', 'skincare', 'protection']
type='Peeling Master Pad' keyword=['exfoliation', 'soothing', 'sensitive skin', 'bio

In [37]:
print(buykorea_preprocessed_dict_openai['Doldori Premium Gold Essence'])

['The Doldori Premium Gold Essence is a unique and innovative mask pack designed for a luxurious skincare experience. Its patented, hygienic design offers a new application method that combines practicality with elegance, making it stand out in the market. This mask pack is available for export and is sold in aesthetic clinics, department store pop-ups, and online shopping malls in Korea, reflecting its high demand.\n\nThe product is manufactured with meticulous attention to quality, from the essence formulation to the mask sheet development. Enriched with high-quality ingredients, the essence hydrates, brightens, and revitalizes the skin, while the mask sheet is designed for excellent adhesion, ensuring comfort and deep absorption of the essence. The Doldori Premium Gold Essence delivers visible skincare benefits along with an indulgent experience, making it a luxurious addition to any skincare routine.', 'mask pack', ['luxurious skincare', 'hygienic design', 'high-quality ingredients

In [38]:
buykorea_preprocessed_openai=pd.DataFrame.from_dict(buykorea_preprocessed_dict_openai, orient='index', columns=['description_summary', 'type', 'keyword'])
buykorea_preprocessed_openai.reset_index(inplace=True)
buykorea_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_openai.to_csv('buykorea_preprocessed_openai.csv', index=False)

display(buykorea_preprocessed_openai.head())

,product_name,description_summary,type,keyword
0,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is a unique a...,mask pack,"[luxurious skincare, hygienic design, high-qua..."
1,Exosome Exo-V Skin Boos,The Exosome Exo-V Skin Booster is a versatile ...,Skin Treatment,"[exosomes, skin regeneration, anti-aging, mois..."
2,Oil-Free Ultra-Moisturizing Lotion with Birch Sap,The Oil-Free Ultra-Moisturizing Lotion with Bi...,lotion,"[oil-free, moisturizing, soothing, hydration, ..."
3,CHARNE - Shiny Brightening Cream,The CHARNE Shiny Brightening Cream is a dual-f...,cosmetic cream,"[skin whitening, wrinkle improvement, natural ..."
4,DEAR OHNEUL Singreen Donkey Toner Pad,The DEAR OHNEUL Singreen Donkey Toner Pad is a...,Toner Pad,"[hypoallergenic, dual-sided, moisturizing, eco..."


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ 업무 배경 설명 (수입자, 공급자, 상품 설명이 존재함을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 공급자와 상품 이름 정보에 기반한 제품 설명 생성
+ 인터넷 검색으로 추가 정보 검색 지시
+ 의미가 확실치 않은 경우 생성 제한
+ 지시 내용별 예시 제공 (one-shot)

In [51]:
generate_prompt_bldata = PromptTemplate.from_template(
    """You are given data where IMP_CO_NAME imports products (PRDT_DC_VAL) from SUPLY_CO_NAME.
    All products fall under HS Code 330499, which is a subcategory of 3304.
    The description for 3304 is as follows:
    33.04 - Beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or sun tan preparations; manicure or pedicure preparations.

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DC_VAL using both SUPLY_CO_NAME and PRDT_DC_VAL.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Example:
    PRDT_DC_VAL: COSMELAN 2 (FACE CREAM)
    SUPLY_CO_NAME: MESOESTETIC, S.L.
    Output Example: This product is called COSMELAN 2, a face cream designed to reduce pigmentation and improve skin tone. It is often used for treating melasma and other skin discolorations by inhibiting melanin production.
    Extract up to {keyword_num} key keywords that describe the product's main features.
    Output Example: pigmentation, skin tone, melasma, melanin, face cream
    Identify the type of product.
    Output Example: face cream
    Extract only the product name from PRDT_DC_VAL, excluding any volume, symbols, special characters, or colors.
    Output Example: COSMELAN 2

    PRODUCT DESCRIPTION(PRDT_DC_VAL) : {prdt_desc}
    SUPPLEYER NAME(SUPLY_CO_NAME) : {suply_name}
    """
)

generate_prompt_bldata= generate_prompt_bldata.partial(keyword_num=keyword_num)

generate_chain_bldata = generate_prompt_bldata | chat_openai

In [52]:
bldata_preprocessed_dict_openai={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = generate_chain_bldata.invoke({"prdt_desc": row['PRDT_DC_VAL'], "suply_name":row['SUPLY_CO_NAME']})
  if index%100==0:
    print(f"\nProduct Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata.content, '\n')
  bldata_preprocessed_dict_openai[row['PRDT_DC_VAL']]=[response_bldata.content]

1it [00:01,  1.86s/it]


Product Name : BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM) 

This product is called BODYSHOCK LOCAL REDUCER, a face cream designed to target localized fat and improve the appearance of the skin. It is often used in body contouring treatments to help reduce the appearance of cellulite and promote smoother skin texture.

**Keywords:** localized fat, cellulite, body contouring, skin texture, face cream

**Type of product:** face cream

**Product name:** BODYSHOCK LOCAL REDUCER 



101it [03:43,  1.80s/it]


Product Name : AVEENO BABY DERMEXA MOISTURING CREAM 206G(1 PAC X 12 PCS) 

This product is called AVEENO BABY DERMEXA MOISTURIZING CREAM, a moisturizing cream specifically formulated for babies to help relieve dry, itchy skin and maintain skin hydration. It is designed to soothe and protect sensitive skin, making it ideal for daily use on infants.

**Key Features:**
1. moisturizing
2. dry skin relief
3. sensitive skin
4. baby care
5. skin hydration

**Type of Product:** moisturizing cream

**Product Name:** AVEENO BABY DERMEXA MOISTURIZING CREAM 



201it [07:07,  1.72s/it]

...

2801it [1:36:58,  1.94s/it]


Product Name : SWISS IMAGE BRAND - SI EN SOOTH.CLEAN.MILK 200ML 

This product is called SWISS IMAGE BRAND - SI EN SOOTH.CLEAN.MILK, a soothing cleansing milk designed to gently remove impurities and makeup from the skin while maintaining its natural moisture balance. It is often formulated with nourishing ingredients to provide a calming effect, making it suitable for sensitive skin types.

**Key Features:**
1. soothing
2. cleansing
3. moisturizing
4. gentle
5. sensitive skin

**Type of Product:** cleansing milk

**Product Name:** SWISS IMAGE BRAND - SI EN SOOTH.CLEAN.MILK 



2901it [1:40:10,  1.82s/it]


Product Name : FACE CARE PRODUCTS OTHER THAN FACE MASK - NS00903 NASHI ARGAN - THE BALM - MOISTURIZING AFTER SHAVE, 100 ML 

This product is called NASHI ARGAN - THE BALM, a moisturizing after shave designed to hydrate and soothe the skin after shaving. It contains argan oil, which is known for its nourishing properties, making it ideal for post-shave care to prevent irritation and dryness.

**Keywords:** moisturizing, after shave, argan oil, hydrate, soothe

**Type of product:** after shave balm

**Product name:** NASHI ARGAN - THE BALM 



2932it [1:41:06,  2.07s/it]


In [56]:
joblib.dump(bldata_preprocessed_dict_openai, 'bldata_preprocessed_dict_openai.pkl')

['bldata_preprocessed_dict_openai.pkl']

In [108]:
bldata_preprocessed_dict_openai_ext=joblib.load('bldata_preprocessed_dict_openai.pkl')

In [109]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"type": {"description": "product type", "title": "Type", "type": "string"}, "keyword": {"description": "5 keywords which focus on product\'s strong point and purpose", "items": {"type": "string"}, "title": "Keyword", "type": "array"}}, "required": ["type", "keyword"]}\n```'

In [110]:
extract_prompt_bldata = PromptTemplate.from_template(
    """Extract product type and keywords from input, based on following rules:
    - Strictly exclude any information about capacity or volume.

    INPUT : {input}
    FORMAT : Extract product type and keywords from input, based on following rules:
    - Strictly exclude any information about capacity or volume.

    INPUT : {input}
    FORMAT : {{ "type": "<product type>", "keyword": ["<keyword1>", "<keyword2>", "<keyword3>", "<keyword4>", "<keyword5>"] }}
    """
)

# extract_prompt_bldata= extract_prompt_bldata.partial(format=parser.get_format_instructions())

extract_chain_bldata = extract_prompt_bldata | chat_openai | parser

In [112]:
for i, (key, value) in tqdm(enumerate(bldata_preprocessed_dict_openai_ext.items())):
  response_bldata = extract_chain_bldata.invoke({"input": value[0]})
  if i%100==0:
    print(response_bldata)
  bldata_preprocessed_dict_openai_ext[key].append(response_bldata.type)
  bldata_preprocessed_dict_openai_ext[key].append(response_bldata.keyword)

1it [00:00,  1.24it/s]

type='face cream' keyword=['localized fat', 'cellulite', 'body contouring', 'skin texture', 'face cream']


101it [01:56,  1.83s/it]

type='highlighter palette' keyword=['luminous glow', 'complexion enhancement', 'multiple shades', 'highlighting', 'sculpted look']


201it [03:39,  1.08it/s]

...

type='cosmetics raw materials' keyword=['cosmetics', 'raw materials', 'formulation', 'emulsifiers', 'preservatives']


2701it [42:26,  1.38it/s]

type='cosmetics' keyword=['natural', 'sun protection', 'cooling effects', 'skin care', 'beauty enhancer']


2801it [44:00,  1.06it/s]

type='moisturizer' keyword=['natural moisturizer', 'vitamins', 'fatty acids', 'hydrating', 'nourishing']


In [121]:
bldata_preprocessed_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai_ext, orient='index', columns=['product_description', 'type', 'keyword'])
bldata_preprocessed_openai.reset_index(inplace=True)
bldata_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_openai.to_csv('bldata_preprocessed_openai.csv', index=False)

display(bldata_preprocessed_openai.head())

,product_name,product_description,type,keyword
0,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),This product is called BODYSHOCK LOCAL REDUCER...,face cream,"[localized fat, cellulite, body contouring, sk..."
1,COSMELAN 2 (FACE CREAM),"This product is called COSMELAN 2, a face crea...",face cream,"[pigmentation, skin tone, melasma, melanin, fa..."
2,ANTI STRESS FACE MASK 100ML (FACE CREAM),"This product is called ANTI STRESS FACE MASK, ...",face cream,"[hydration, soothing, skin relief, reduce redn..."
3,BLOT POWDER,"This product is called BLOT POWDER, a setting ...",setting powder,"[shine control, oil absorption, matte finish, ..."
4,DERMACOLOR CAMOUFLAGE FLUID,This product is called DERMACOLOR CAMOUFLAGE F...,camouflage fluid,"[high coverage, skin imperfections, long-lasti..."


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


# 수출한 기업은 다르지만, 수출한 품목의 이름은 같은 경우가 있어 불일치 발생
(중복 제거됨)

In [115]:
# 같은 품목 중 다른 기업이 수출한 이력이 있는 경우, 다른 수출건으로 집계됨

bl_data_sampled.shape

(2932, 3)

In [116]:
# 제품 이름에 기반해, 제품 수입 확률이 높은 해외 바이어 매칭이므로
# 중복되는 제품 이름은 하나의 키워드 및 임베딩으로 통일

len(bldata_preprocessed_dict_openai)

2801

# 임베딩

In [117]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [118]:
def text_embedding(text):
  embedding_text=embeddings.embed_query(text)
  return np.array(embedding_text).reshape(1,-1)

#  1. 키워드 10개로 이루어진 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['type']+":"+buykorea_preprocessed_openai['purpose']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['type']+":"+bldata_preprocessed_openai['purpose']
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_keyword', 'bldata_product_name', 'bldata_keyword', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_keyword': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())

In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v1.csv', index=False)

# 2. 제품 유형만 이용해 유사도 순 나열

In [123]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['type']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['type']
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [04:20<00:00, 10.77it/s]


In [124]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'bldata_product_name', 'bldata_type', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_type']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:05, 507.90it/s]
2801it [00:04, 568.80it/s]
2801it [00:04, 600.25it/s]
2801it [00:05, 500.14it/s]
2801it [00:04, 591.45it/s]
2801it [00:04, 565.15it/s]
2801it [00:05, 492.79it/s]
2801it [00:04, 595.83it/s]
2801it [00:05, 496.83it/s]
2801it [00:04, 577.05it/s]


,buykorea_product_name,buykorea_type,bldata_product_name,bldata_type,cosine_similarity
0,Doldori Premium Gold Essence,mask pack,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),face cream,0.443385
1,Doldori Premium Gold Essence,mask pack,COSMELAN 2 (FACE CREAM),face cream,0.443385
2,Doldori Premium Gold Essence,mask pack,ANTI STRESS FACE MASK 100ML (FACE CREAM),face cream,0.443385
3,Doldori Premium Gold Essence,mask pack,BLOT POWDER,setting powder,0.142408
4,Doldori Premium Gold Essence,mask pack,DERMACOLOR CAMOUFLAGE FLUID,camouflage fluid,0.390623


In [125]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v2.csv', index=False)

,buykorea_product_name,buykorea_type,bldata_product_name,bldata_type,cosine_similarity,test_row
0,Doldori Premium Gold Essence,mask pack,AVOCADO HIGH NUTRITION MASK PACK (BRAND-STAROV...,mask pack,1.000000,Highest
1,Doldori Premium Gold Essence,mask pack,COSMETICS MASK PACK,mask pack,1.000000,Highest
2,Doldori Premium Gold Essence,mask pack,ASSORTD FACE PACK (RETURN TO ORIGIN) (EXPORTED...,face pack,0.785882,Highest
3,Doldori Premium Gold Essence,mask pack,SS0000016020SS APC MASK 13G TUBE,skincare mask,0.764832,Highest
4,Doldori Premium Gold Essence,mask pack,HIBISCUS & ROSE GLOW MASK (RETURN TO ORIGIN) (...,skincare mask,0.764832,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,sunscreen,HILARY RHODA HIGHLIGHTER & CONTOUR (NT. WT. 12...,highlighter and contour,0.045261,Lowest
28006,t:aim Barrier Sunblock 40ml,sunscreen,PEPPLUS LIFTING PACK (8 PACKS) ( 40 PCS) ( CDS...,lifting pack,0.041898,Lowest
28007,t:aim Barrier Sunblock 40ml,sunscreen,LYON BEAUTY USA - PRIMA VISTA MULTI COLOR BASE...,multi-color base,0.038602,Lowest
28008,t:aim Barrier Sunblock 40ml,sunscreen,2.14725 YURAQ (SAMPLE-NCV-R&D PURPOSE) (1 X 25...,sample for research and development,0.033819,Lowest


# 3. 제품 유형 + 키워드 5개 (총 6개 키워드) 이용해 유사도 순 나열

In [134]:
buykorea_preprocessed_openai['keyword'][0]

"['luxurious skincare', 'hygienic design', 'high-quality ingredients', 'hydration', 'skin revitalization']"

In [ ]:
# 1. 양쪽 대괄호 제거 및 마지막 쉼표 제거
cleaned_string = string.strip('[]').rstrip(',')

# 2. 쉼표를 기준으로 분리하여 리스트로 변환
result_list = cleaned_string.split(',')

In [136]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['type']+","+buykorea_preprocessed_openai['keyword'].apply(lambda x : ",".join(x.strip('[]').split(',')))
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['type']+","+bldata_preprocessed_openai['keyword'].apply(lambda x : ",".join(x.strip('[]').split(',')))
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [08:44<00:00,  5.34it/s]


In [137]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_keyword', 'bldata_product_name', 'bldata_keyword', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_keyword': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:05, 530.75it/s]
2801it [00:04, 582.21it/s]
2801it [00:05, 482.78it/s]
2801it [00:04, 588.38it/s]
2801it [00:05, 552.71it/s]
2801it [00:05, 510.13it/s]
2801it [00:04, 575.73it/s]
2801it [00:05, 482.47it/s]
2801it [00:04, 577.53it/s]
2801it [00:04, 582.27it/s]


,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),"face cream,'localized fat', 'cellulite', 'body...",0.573740
1,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",COSMELAN 2 (FACE CREAM),"face cream,'pigmentation', 'skin tone', 'melas...",0.474314
2,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",ANTI STRESS FACE MASK 100ML (FACE CREAM),"face cream,'hydration', 'soothing', 'skin reli...",0.667383
3,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",BLOT POWDER,"setting powder,'shine control', 'oil absorptio...",0.645117
4,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",DERMACOLOR CAMOUFLAGE FLUID,"camouflage fluid,'high coverage', 'skin imperf...",0.603760


In [138]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v3.csv', index=False)

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity,test_row
0,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",COSMETICS MASK,"mask,'hydration', 'nourishment', 'rejuvenation...",0.850780,Highest
1,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",SS0000014020 SS APC MASK 30G TRAVEL SIZE FILLE...,"facial mask,'skin care', 'hydration', 'easy ap...",0.830218,Highest
2,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",BELIF THE TRUE CREAM MOISTURIZING BOMB 25ML (C...,"moisturizer,'intense moisture', 'herbal ingred...",0.823652,Highest
3,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",OAP RV CRYSTAL MASK 1PC AS (OAP RV CRYSTAL MAS...,"skincare mask,'hydration', 'rejuvenation', 'sk...",0.819121,Highest
4,Doldori Premium Gold Essence,"mask pack,'luxurious skincare', 'hygienic desi...",COSMETICS DR.JART CRYO RUBBER WITH MOISTURIZIN...,"moisturizing mask,'moisturizing', 'hyaluronic ...",0.816542,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF50+', 'UV pro...",COLOR ICON EYESHADOW GLITTER SINGLE(1.4gm)(COS...,"eyeshadow,'eyeshadow', 'glitter', 'cosmetic', ...",0.305137,Lowest
28006,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF50+', 'UV pro...",EYEBROW PENCIL,"eyebrow pencil,'define', 'enhance', 'shape', '...",0.303874,Lowest
28007,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF50+', 'UV pro...",COSMETICS- SUGAR Arch Arrival 3 in 1 Brow Shap...,"brow shaper,'brow shaper', 'define', 'fill', '...",0.281756,Lowest
28008,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF50+', 'UV pro...",POLYESTER GLITTER,"glitter,'glitter', 'cosmetic', 'sparkle', 'mak...",0.261830,Lowest


# 4. 유형 단어 포함 여부로 필터 후, 키워드 5개 이용해 유사도 순 나열

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['purpose']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['purpose']
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_keyword', 'bldata_product_name', 'bldata_keyword', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_keyword': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  buykorea_product_type=row['type'].split(' ')

  pattern = '|'.join(buykorea_product_type) + '|' + ' '.join(buykorea_product_type)
  result_pattern = f'({pattern})'

  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):
    if row_bl['type'].str.contains(result_pattern, Regex=True):
      bldata_embedding=row_bl['bldata_embedding']

      similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
      similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
      similarity_df.loc[row_count, 'bldata_keyword']=row_bl['bldata_preprocessed']
      similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v4.csv', index=False)

# 5. 유형 유사도 순 나열 후, 비슷한 유형 내에서 키워드 5개 이용해 유사도 순 나열

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_type_preprocessed']=buykorea_preprocessed_openai['type']
bldata_preprocessed_openai['bldata_type_preprocessed']=bldata_preprocessed_openai['type']
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['buykorea_type_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['bldata_type_preprocessed'].progress_apply(text_embedding)

buykorea_preprocessed_openai['buykorea_keyword_preprocessed']=buykorea_preprocessed_openai['keyword']
bldata_preprocessed_openai['bldata_keyword_preprocessed']=bldata_preprocessed_openai['keyword']
# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_keyword_embedding']=buykorea_preprocessed_openai['buykorea_keyword_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_keyword_embedding']=bldata_preprocessed_openai['bldata_keyword_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_keyword', 'bldata_product_name', 'bldata_type', 'bldata_keyword', 'type_cosine_similarity', 'keyword_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                              'buykorea_type' : [row['buykorea_type_preprocessed']]*bldata_sample_len,
                               'buykorea_keyword': [row['buykorea_keyword_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                              'keyword_cosine_similarity ' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_keyword_embedding=row['buykorea_keyword_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_keyword_embedding=row_bl['bldata_keyword_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    keyword_similarity=cosine_similarity(buykorea_keyword_embedding, bldata_keyword_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_type']=row_bl['bldata_type_preprocessed']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['bldata_keyword_preprocessed']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'keyword_cosine_similarity']=keyword_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(subset=['type_cosine_similarity','keyword_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v5.csv', index=False)